# AIFFEL 여섯번째 프로젝트 : 멋진 작사가 만들기
---
## 1. 데이터 다운로드
```
$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기
```
## 2. 데이터 읽어오기
- glob 모듈을 사용해 모든 txt 파일 읽고, raw_corpus 리스트에 문장 단위로 저장하기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['They say get ready for the revolution', "I think it's time we find some sorta solution", "Somebody's caught up in the endless pollution"]


## 3. 데이터 정제
- `preprocess_sentence()` 함수를 활용해 데이터 정제하기
    - 지나치게 긴 문장은 다른 데이터들이 과도한 padding을 갖게 하므로 제거
    - \+ 노래가사 작사에 그닥 어울리지 않음
- **문장을 토큰화 했을 때, 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권장** 

In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

def preprocess_sentence(sentence):
    
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿,:])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()
    
    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니        
        
    return sentence

result = preprocess_sentence("You: I, am a this is my wish that nobodies neglected be like a rocket baby:")
print(result)# 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> you i , am a this is my wish that nobodies neglected be like a rocket baby <end>


In [3]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0 : continue
    # 문장 중 토큰화했을 때 개수 15개 넘으면 삭제(최대 14개)
    temp = preprocess_sentence(sentence)
    if len(temp.split(" ")) > 15: continue

    corpus.append(temp)
len(corpus)

156227

## 4. 평가 데이터셋 분리
1. `tokenize()`로 데이터를 텐서로 변환
2. `sklearn` 모듈의 `train_test_split()`를 이용해 데이터 분리
    - 단어장 크기 12000 이상으로 설정 권장
    - 총 데이터의 20%를 평가 데이터셋으로 사용 권장  
    
`enc_train, enc_val, dec_train, dec_val = <코드작성>`

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
len(tensor)

[[  2  38  71 ...   0   0   0]
 [  2   4 130 ...   0   0   0]
 [  2 247  17 ...   0   0   0]
 ...
 [  2  33   7 ...   0   0   0]
 [  2  13 436 ...  10  12   3]
 [  2  26  15 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f7e86702e90>


156227

In [5]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

In [25]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 34)

BUFFER_SIZE = len(enc_train)
BUFFER_SIZE_val = len(enc_val)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

validation = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE_val)
validation = validation.batch(BATCH_SIZE, drop_remainder=True)

dataset,validation

(<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>,
 <BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>)

In [14]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


## 5. 인공지능 만들기
- 모델의 Embedding Size, Hidden Size 조절하며 10 Epoch 안에 val_loss 2.2 수준으로 줄일 수 있는 모델 설계하기!  
- Loss 함수 : `loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')`

In [30]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
#         self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
#         out = self.rnn_3(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [31]:
# 모델 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=validation)

Epoch 1/10
488/488 [==============================] - 233s 477ms/step - loss: 3.2024 - val_loss: 2.8430
Epoch 2/10
488/488 [==============================] - 219s 450ms/step - loss: 2.6680 - val_loss: 2.6197
Epoch 3/10
488/488 [==============================] - 219s 450ms/step - loss: 2.3460 - val_loss: 2.4227
Epoch 4/10
488/488 [==============================] - 220s 450ms/step - loss: 2.0315 - val_loss: 2.2965
Epoch 5/10
488/488 [==============================] - 220s 451ms/step - loss: 1.7428 - val_loss: 2.2061
Epoch 6/10
488/488 [==============================] - 219s 450ms/step - loss: 1.4941 - val_loss: 2.1530
Epoch 7/10
488/488 [==============================] - 219s 449ms/step - loss: 1.2963 - val_loss: 2.1315
Epoch 8/10
488/488 [==============================] - 218s 448ms/step - loss: 1.1516 - val_loss: 2.1329
Epoch 9/10
488/488 [==============================] - 218s 447ms/step - loss: 1.0576 - val_loss: 2.1499
Epoch 10/10
488/488 [==============================] - 218s 447m

In [32]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [34]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you liberian girl , <end> '

---
# 루브릭
## 1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
- 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
- 문장 생성 함수의 결과로 `<start> i love you liberian girl , <end>`의 그럴듯한 문장을 생성했다.

## 2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
- 특수문자 제거, 토크나이저 생성, 패딩 처리 등의 과정이 빠짐없이 진행되었는가?
- 전처리 함수 `def preprocess_sentence(sentence)`를 통해 특수문자 제거를 진행했다.
    - `sentence = sentence.lower().strip()` : 소문자로 바꾸고 양쪽 공백을 삭제
    - `sentence = re.sub(r"([?.!,¿,:])", r" \1 ", sentence)` : 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    - `sentence = re.sub(r'[" "]+', " ", sentence)` : 공백 패턴을 만나면 스페이스 1개로 치환
    - `sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)` : a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    - `sentence = '<start> ' + sentence + ' <end>'` : 이전 스텝에서 본 것처럼 문장 앞뒤로 \<start>와 \<end>를 추가
    - 문장 생성 후, `if len(temp.split(" ")) > 15: continue` : 토큰화 후 토큰 개수가 15개 넘는 것은 데이터에서 제거
- 토크나이저 생성과 패딩 처리를 위해, `def tokenize(corpus)` 함수를 사용하였다.
    - `tokenizer = tf.keras.preprocessing.text.Tokenizer()`에 전처리된 문장을 적용하여 모델에 사용할 단어 사전을 만든다.
    - `tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)` : 패딩 방식은 post 방식을 이용하였고, maxlen은 토큰 개수 제한을 바탕으로 maxlen을 설정하였다. 앞서 전처리 과정에서 15 이상은 제거되었기 때문에, 최대 길이 15로 설정되어 굳이 쓰지 않아도 된다.
    
## 3. 텍스트 생성모델이 안정적으로 학습되었는가?
- 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?
    - 처음 진행했을 당시 **dataset 변경 과정을 거치지 않은 실수**를 해서 embedding size, hidden size가 노드의 2배만 되어도, 학습 과정이 굉장히 길어졌다. 그리고 loss 값이 embedding size, hidden size를 조절해도 2.4 이하로 떨어지지 않았다. 조원분들께 여쭤보고 나서 오류를 발견하게 되어 dataset 으로 변경하는 과정을 거쳤다. dataset 타입으로 변경하는 과정만 넣었을 뿐인데 loss 값이 데이터 타입을 바꾸기 전에 비해 0.3~0.5 가량 떨어졌고, 학습 진행 속도도 훨씬 빨라졌다. 데이터 타입이 적절하지 않을 때의 **코드 오류(단순한 에러)** 만 생각했어서, 실수 과정에서 어쩌다 보니 디버깅이 필요했던 실수를 했다. 프로젝트하는 시간이 많이 할애되었지만, 학습이 비정상적으로 진행될 때 **데이터 타입(혹은 차원)의 문제**를 생각할 수 있게 되었다.

---
# 마무리  
2021 01 28 THUR
- 다른 노드에 비해 이해하기 쉬운 편이었고, E4를 어렵게 풀어 비교적 쉽게 느껴졌을 수도... val_loss의 레벨을 낮추기 위해 embedding size, hidden size를 크게 했을 때, 모델 학습이 많이 느리다는 점이 노드의 가장 큰 걸림돌이었다. 그리고 중간에 데이터 변환을 하지 않는 실수를 해서 학습 과정이 비정상적이면 어느 과정에서 오류가 났을지에 대한 카테고리가 하나 더 생겼다. ^^..
- AIFFEL Exploration에서 항상 데이터의 중요성을 이야기한다. 이미지 데이터로 활용할 수 있는게 많지만 문자 데이터로도 전달할 수 있는게 많을 것 같다. **E4와 연동하면, E5에서 문장을 만들고 E4처럼 단어를 말해주는 E-book 컨텐츠로 활용하거나, 반대로 E4의 음성을 파악해 E5를 적용한 자막 만들기도 하면 재밌을 것 같다.**